In [ ]:
#Setup and Installation
!nvidia-smi
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting.git
%cd gaussian-splatting

!pip install -q plyfile tqdm pillow opencv-python
!pip install -q submodules/diff-gaussian-rasterization
!pip install -q submodules/simple-knn

!apt-get install -y colmap imagemagick ffmpeg

import torch
import os
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

Sat Jan  3 11:19:18 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

import zipfile
import os

# Paths
zip_path = "/content/drive/MyDrive/Dataset/nerf_synthetic.zip"
extract_path = "/content/data"

# Create extract directory
os.makedirs(extract_path, exist_ok=True)

# Extract
print("Extracting NeRF Synthetic dataset")
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

# Verify
!ls /content/data/


ValueError: mount failed

In [ ]:
!ls

In [ ]:
!ls /content/data/lego

In [ ]:
import os
import json
from PIL import Image
!pwd
def load_images_from_folder(folder):
    images = []
    for filename in sorted(os.listdir(folder)):
        if filename.endswith(".png") and "depth" not in filename:
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path).convert("RGB")
            images.append(img)
    return images

def load_depths_from_folder(folder):
    depths = []
    for filename in sorted(os.listdir(folder)):
        if filename.endswith(".png") and "depth" in filename:
            depth_path = os.path.join(folder, filename)
            depth_img = Image.open(depth_path)
            depths.append(depth_img)
    return depths

def load_metadata(json_path):
    with open(json_path, "r") as f:
        data = json.load(f)
    return data

# Paths
base_path = "/content/data/lego"
train_folder = os.path.join(base_path, "train")
val_folder = os.path.join(base_path, "val")
test_folder = os.path.join(base_path, "test")
json_path = os.path.join(base_path, "transforms_val.json")

# Load
train_images = load_images_from_folder(train_folder)
train_depths = load_depths_from_folder(train_folder)
metadata = load_metadata(json_path)

print(f"Loaded {len(train_images)} training images and {len(train_depths)} depth maps")
print(f"Metadata keys: {list(metadata.keys())}")


In [ ]:
# CELL 3: Convert Dataset
# SCENE_NAME = 'lego'
# SOURCE_PATH = f'data/{SCENE_NAME}'

# !python convert.py -s {SOURCE_PATH}

# print(f"Data converted for scene: {SCENE_NAME}")
# !ls {SOURCE_PATH}

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

SCENE_NAME = 'lego'
SOURCE_PATH = f'/content/data/{SCENE_NAME}'

# Use the 'train' folder directly (no images/ folder)
train_images_path = os.path.join(SOURCE_PATH, 'train')

# Get image files (PNG or JPG) from train folder
image_files = sorted([f for f in os.listdir(train_images_path) if f.endswith(('.png', '.jpg'))])[:8]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for idx, img_file in enumerate(image_files):
    img_path = os.path.join(train_images_path, img_file)
    img = Image.open(img_path)
    axes[idx].imshow(img)
    axes[idx].set_title(img_file)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print(f"Total training images: {len(os.listdir(train_images_path))}")


In [ ]:
# CELL 5: Train Model (Quick - 7000 iterations)
import time

MODEL_PATH = f'output/models/{SCENE_NAME}'
ITERATIONS = 7000

os.makedirs(MODEL_PATH, exist_ok=True)

train_cmd = f"""
python train.py \
    -s {SOURCE_PATH} \
    -m {MODEL_PATH} \
    --iterations {ITERATIONS} \
    --test_iterations 1000 3000 7000 \
    --save_iterations 1000 3000 7000 \
    --checkpoint_iterations 1000 3000 7000 \
    --resolution 1 \
    --eval
"""

print("Starting training...")
start_time = time.time()
!{train_cmd}
end_time = time.time()

print(f"\nTraining completed in {(end_time - start_time) / 60:.2f} minutes")

In [ ]:
# CELL 6: Render Test Views
RENDER_PATH = f'output/renders/{SCENE_NAME}'

render_cmd = f"""
python render.py \
    -m {MODEL_PATH} \
    --skip_train \
    --iteration 7000
"""

print("Rendering test views...")
!{render_cmd}

print(f"Renders saved to: {MODEL_PATH}/test/ours_7000/renders")

In [ ]:
# CELL 7: Evaluate Model
eval_cmd = f"""
python metrics.py \
    -m {MODEL_PATH}
"""

print("Evaluating model...")
!{eval_cmd}

In [ ]:
# CELL 8: Visualize Results
import json
from PIL import Image

renders_path = f"{MODEL_PATH}/test/ours_7000/renders"
gt_path = f"{MODEL_PATH}/test/ours_7000/gt"

render_files = sorted(os.listdir(renders_path))[:8]

fig, axes = plt.subplots(4, 4, figsize=(16, 16))

for idx, img_file in enumerate(render_files):
    render_img = Image.open(os.path.join(renders_path, img_file))
    gt_img = Image.open(os.path.join(gt_path, img_file))

    axes[idx*2//4, (idx*2)%4].imshow(render_img)
    axes[idx*2//4, (idx*2)%4].set_title(f'Rendered {idx}')
    axes[idx*2//4, (idx*2)%4].axis('off')

    axes[idx*2//4, (idx*2)%4 + 1].imshow(gt_img)
    axes[idx*2//4, (idx*2)%4 + 1].set_title(f'Ground Truth {idx}')
    axes[idx*2//4, (idx*2)%4 + 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
!pwd

In [ ]:
# CELL 9: Create Video from Renders
video_path = f"output/videos/{SCENE_NAME}_test.mp4"
os.makedirs(os.path.dirname(video_path), exist_ok=True)

ffmpeg_cmd = f"""
ffmpeg -y -framerate 30 -pattern_type glob -i '{renders_path}/*.png' \
    -c:v libx264 -pix_fmt yuv420p {video_path}
"""

!{ffmpeg_cmd}

print(f"Video saved to: {video_path}")

from IPython.display import Video
Video(video_path, width=800)

In [ ]:
# CELL 10: Load Metrics
results_file = f"{MODEL_PATH}/results.json"

if os.path.exists(results_file):
    with open(results_file, 'r') as f:
        results = json.load(f)

    print("Evaluation Metrics:")
    print("=" * 50)
    for key, value in results.items():
        if isinstance(value, dict):
            print(f"\n{key}:")
            for k, v in value.items():
                print(f"  {k}: {v}")
        else:
            print(f"{key}: {value}")
    print("=" * 50)
else:
    print("Results file not found. Run evaluation first.")

In [ ]:
# CELL 11: Render Custom Camera Path (360 rotation)
custom_render_path = f"output/renders/{SCENE_NAME}_360"
os.makedirs(custom_render_path, exist_ok=True)

render_360_cmd = f"""
python render.py \
    -m {MODEL_PATH} \
    --iteration 7000 \
    --skip_train \
    --skip_test
"""

!{render_360_cmd}

print("Custom camera path rendered")

In [ ]:
# CELL 12: Train Full Quality (Optional - 30000 iterations)
FULL_MODEL_PATH = f'/content/output/models/{SCENE_NAME}_full'
FULL_ITERATIONS = 30000

os.makedirs(FULL_MODEL_PATH, exist_ok=True)

full_train_cmd = f"""
python train.py \
    -s {SOURCE_PATH} \
    -m {FULL_MODEL_PATH} \
    --iterations {FULL_ITERATIONS} \
    --test_iterations 7000 15000 30000 \
    --save_iterations 7000 15000 30000 \
    --checkpoint_iterations 7000 15000 30000 \
    --resolution 1 \
    --eval
"""

print("Starting full training (30000 iterations)...")
print("This will take 20-30 minutes")

start_time = time.time()
!{full_train_cmd}
end_time = time.time()

print(f"\nFull training completed in {(end_time - start_time) / 60:.2f} minutes")

In [ ]:
# CELL 13: Process Custom Images (Colab version)
from google.colab import files
import os
import shutil

CUSTOM_PATH = "data/custom_scene"
INPUT_PATH = os.path.join(CUSTOM_PATH, "input")
os.makedirs(INPUT_PATH, exist_ok=True)

# Upload images directly from your computer
print("Upload your images (JPG/PNG) for the custom scene:")
uploaded_files = files.upload()  # will open upload dialog

# Move uploaded files to input folder
for filename in uploaded_files.keys():
    shutil.move(filename, os.path.join(INPUT_PATH, filename))

print(f"Uploaded {len(uploaded_files)} images to {INPUT_PATH}")

# Optional: Run convert.py if needed (adjust if your dataset needs conversion)
# !python convert.py -s {CUSTOM_PATH} --colmap_executable colmap


In [ ]:
!pwd

In [ ]:
import os

MODEL_PATH = '/content/output/models/lego_full'
CFG_PATH = os.path.join(MODEL_PATH, 'cfg_args')

cfg_text = """__import__('types').SimpleNamespace(
    scene='lego',
    source_path='/content/data/lego',
    resolution=1,
    iterations=30000,
    eval=True,
    device='cuda'
)"""

with open(CFG_PATH, 'w', encoding='utf-8') as f:
    f.write(cfg_text)

print(f"SimpleNamespace cfg_args created at: {CFG_PATH}")


In [ ]:
# Paths
SCENE_NAME = 'custom_scene'
SOURCE_PATH = f'/content/data/{SCENE_NAME}'
MODEL_PATH = f'/content/output/models/lego_full'
OUTPUT_PATH = os.path.join(MODEL_PATH, 'rendered')
os.makedirs(OUTPUT_PATH, exist_ok=True)

#Create cfg_args if missing
CFG_PATH = os.path.join(MODEL_PATH, 'cfg_args')
if not os.path.exists(CFG_PATH):
    cfg_text = """__import__('types').SimpleNamespace(
        scene='lego',
        source_path='/content/data/lego',
        resolution=1,
        iterations=30000,
        eval=True,
        device='cuda'
    )"""
    with open(CFG_PATH, 'w', encoding='utf-8') as f:
        f.write(cfg_text)
    print(f"Created cfg_args at {CFG_PATH}")

# Find the latest checkpoint automatically
checkpoint_files = sorted(glob.glob(os.path.join(MODEL_PATH, "*.pth")))
if checkpoint_files:
    latest_checkpoint = checkpoint_files[-1]
    # Extract digits from filename safely
    match = re.search(r'\d+', os.path.basename(latest_checkpoint))
    if match:
        CHECKPOINT = int(match.group())
    else:
        CHECKPOINT = 30000
    print(f"Using latest checkpoint: {latest_checkpoint} (iteration {CHECKPOINT})")
else:
    CHECKPOINT = 30000
    print(f"No checkpoint found, using default iteration {CHECKPOINT}")

#  Render 3D image
print("Rendering 3D image from trained model...")
render_image_path = os.path.join(OUTPUT_PATH, 'rendered_image.png')

!python render.py \
    --source_path {SOURCE_PATH} \
    --model_path {MODEL_PATH} \
    --iteration {CHECKPOINT} \
    --resolution 1 \
    --eval

# Display the last rendered image
rendered_images = sorted([f for f in os.listdir(MODEL_PATH) if f.endswith(".png")])
if rendered_images:
    last_image_path = os.path.join(MODEL_PATH, rendered_images[-1])
    print(f"Displaying rendered image: {last_image_path}")
    display(Image(last_image_path))
else:
    print("No rendered image found. Check render.py output paths.")


In [ ]:
#Process Video to Images
VIDEO_PATH = "data/custom_video/video.mp4"
VIDEO_OUTPUT = "data/custom_video/images"

os.makedirs(VIDEO_OUTPUT, exist_ok=True)

video_to_frames_cmd = f"""
ffmpeg -i {VIDEO_PATH} -qscale:v 1 -qmin 1 -vf fps=2 {VIDEO_OUTPUT}/%04d.jpg
"""

print("Upload your video first, then run:")
print(video_to_frames_cmd)

In [ ]:
#Export Point Cloud
ply_path = f"{MODEL_PATH}/point_cloud/iteration_7000/point_cloud.ply"

if os.path.exists(ply_path):
    print(f"Point cloud saved at: {ply_path}")

    from plyfile import PlyData
    plydata = PlyData.read(ply_path)

    print(f"Number of points: {plydata['vertex'].count}")
    print(f"Properties: {plydata['vertex'].properties}")
else:
    print("Point cloud not found")

In [ ]:
#Cleanup and Save Results
import shutil

print("Saving important files...")

files_to_keep = [
    f"{MODEL_PATH}/point_cloud/iteration_7000/point_cloud.ply",
    f"{MODEL_PATH}/cameras.json",
    f"{MODEL_PATH}/results.json",
]

for file_path in files_to_keep:
    if os.path.exists(file_path):
        print(f"✓ {file_path}")
    else:
        print(f"✗ {file_path} not found")

print("\nModel training complete!")
print(f"Model saved at: {MODEL_PATH}")
print(f"Renders saved at: {MODEL_PATH}/test/ours_7000/renders")

In [ ]:
#Compare Multiple Scenes
SCENES = ['lego', 'chair', 'drums']

for scene in SCENES:
    source = f'data/nerf_synthetic/{scene}'
    model = f'output/models/{scene}_quick'

    print(f"\nProcessing {scene}...")
    !python convert.py -s {source}
    !python train.py -s {source} -m {model} --iterations 5000 --resolution 2 --eval
    !python render.py -m {model} --skip_train --iteration 5000
    !python metrics.py -m {model}

In [ ]:
# Advanced Training Options
ADVANCED_MODEL_PATH = f'output/models/{SCENE_NAME}_advanced'

advanced_train_cmd = f"""
python train.py \
    -s {SOURCE_PATH} \
    -m {ADVANCED_MODEL_PATH} \
    --iterations 30000 \
    --position_lr_init 0.00016 \
    --position_lr_final 0.0000016 \
    --feature_lr 0.0025 \
    --opacity_lr 0.05 \
    --scaling_lr 0.005 \
    --rotation_lr 0.001 \
    --densify_grad_threshold 0.0002 \
    --densification_interval 100 \
    --opacity_reset_interval 3000 \
    --densify_from_iter 500 \
    --densify_until_iter 15000 \
    --sh_degree 3 \
    --eval
"""

print("Training with custom hyperparameters...")
!{advanced_train_cmd}

In [ ]:
# Batch Render All Test Views
all_renders_path = f"output/all_renders/{SCENE_NAME}"
os.makedirs(all_renders_path, exist_ok=True)

iterations = [1000, 3000, 7000]

for iter_num in iterations:
    print(f"\nRendering iteration {iter_num}...")
    render_cmd = f"""
    python render.py \
        -m {MODEL_PATH} \
        --iteration {iter_num} \
        --skip_train
    """
    !{render_cmd}

    src = f"{MODEL_PATH}/test/ours_{iter_num}/renders"
    dst = f"{all_renders_path}/iter_{iter_num}"

    if os.path.exists(src):
        shutil.copytree(src, dst, dirs_exist_ok=True)
        print(f"Saved to {dst}")

In [ ]:
# Final Summary
print("TRAINING SUMMARY")
print(f"Scene: {SCENE_NAME}")
print(f"Model Path: {MODEL_PATH}")
print(f"Iterations: {ITERATIONS}")
print()

if os.path.exists(f"{MODEL_PATH}/results.json"):
    with open(f"{MODEL_PATH}/results.json", 'r') as f:
        results = json.load(f)
    print("Metrics:")
    for key, value in results.items():
        if not isinstance(value, dict):
            print(f"  {key}: {value}")

print()
print("Output Files:")
print(f"  - Model: {MODEL_PATH}")
print(f"  - Renders: {MODEL_PATH}/test/ours_7000/renders")
print(f"  - Point Cloud: {MODEL_PATH}/point_cloud/iteration_7000/point_cloud.ply")
print("=" * 80)